# Спринт 13 «Линейная алгебра»

# Проект «Защита персональных данных клиентов»

## Навигация

1. [Загрузка данных](#Загрузка-данных)
1. [Умножение матриц](#Умножение-матриц)
1. [Алгоритм преобразования](#Алгоритм-преобразования)
1. [Проверка алгоритма](#Проверка-алгоритма)
   - [Эксперимент без кодирования](#Эксперимент-без-кодирования)
   - [Эксперимент с кодированием](#Эксперимент-с-кодированием)
1. [Общий вывод](#Общий-вывод)

Нам нужно защитить данные клиентов страховой компании «Хоть потоп». Необходимо разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

In [1]:
import os

import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [2]:
RANDOM_STATE = 42

## Загрузка данных

[к навигации](#Навигация)

In [3]:
data = pd.read_csv(os.path.join('..', 'datasets', 'insurance.csv'))
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


__Выводы:__ 
1. Пропуски отсутствуют. Отлично!
2. `Пол` -  бинарный признак, `Возраст` - численный непрерывный, `Зарплата` - численный непрерывный, `Члены семьи` - численный дискретный. Целевой признак `Страховые выплаты` - является численным дискретным.
3. После предсказания линейной регрессией, хорошо бы округлять ответ до целых.

## Умножение матриц

[к навигации](#Навигация)

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:**  
Не изменится


**Обоснование:** 

замена $w$  
$$
a = X (X^T X)^{-1} X^T y
$$



с кодированием ($P$ - матрица кодирования), знаем, что она обратима
$$
a' = (XP) ((XP)^T (XP))^{-1} (XP)^T y
$$

$$
a' = X P ((XP)^T (XP))^{-1} (XP)^T y
$$

$$
a' = X P (P^T X^T X P)^{-1} P^T X^T y
$$

вот здесь не уверен, что правильно раскрываю скобку из под -1 с тремя множителями
$$
a' = X P (P^T (X^T X) P)^{-1} P^T X^T y
$$

$$
a' = X P P^{-1} (X^T X)^{-1} P^{T^{-1}} P^T X^T y
$$

$$
a' = X (PP^{-1}) (X^T X)^{-1} (P^{T^{-1}}P^T) X^T y
$$

$$
a' = X E (X^T X)^{-1} E X^T y
$$

$$
a' = X (X^T X)^{-1} X^T y
$$

$$
a' = a
$$

## Алгоритм преобразования

[к навигации](#Навигация)

**Алгоритм**

- Создать квадратную матрицу со сторонами, равными количеству признаков.
- Проверить, является ли она обратимой.
- Закодировать признаки, умножив их на данную матрицу.

**Обоснование**

Да в целом оно описано в п.2.

## Проверка алгоритма

[к навигации](#Навигация)

In [5]:
X = data.drop(columns='Страховые выплаты')
y = data['Страховые выплаты']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=RANDOM_STATE)

### Эксперимент без кодирования

[к навигации](#Навигация)

In [7]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [8]:
y_pred = model.predict(X_test)
r2_score(y_test, y_pred)

0.4254778540696319

### Эксперимент с кодированием

[к навигации](#Навигация)

In [9]:
n = X.shape[1]
coder = np.random.random((n, n))
decoder = np.linalg.inv(coder)  # выдаст ошибку, если матрица необратима, иначе вернёт обратную матрицу

In [10]:
X_train_c = X_train @ coder
X_test_c = X_test @ coder

In [11]:
model_c = LinearRegression()
model_c.fit(X_train_c, y_train)

LinearRegression()

In [12]:
y_pred_c = model_c.predict(X_test_c)
r2_score(y_test, y_pred_c)

0.4254778540700015

Результыта почти неотличимы. Скорее всего теряется точность при умножении float * float.

In [13]:
X_train_dc = (X_train_c @ decoder).round(0).astype('int')
X_test_dc = (X_test_c @ decoder).round(0).astype('int')

In [14]:
model_dc = LinearRegression()
model_dc.fit(X_train_dc, y_train)

LinearRegression()

In [15]:
y_pred_dc = model_dc.predict(X_test_dc)
r2_score(y_test, y_pred_dc)

0.4254778540696319

In [16]:
# Код ревьюера
a = X_train @ coder @ decoder
a.round(0).astype('int')

,0,1,2,3
4884,0,34,40200,0
3163,0,23,33300,1
490,0,34,43600,0
862,1,36,34600,1
1740,0,36,41000,1
...,...,...,...,...
4426,1,25,36300,2
466,0,33,25700,3
3092,1,38,46500,0
3772,1,33,35900,0


## Общий вывод

[к навигации](#Навигация)

С поставленной задачей справились. Разработанный метод
- защищает персональные данные клиентов,
- не снижает перфоманс моделей машинного обучения.